In [4]:
!pip3 install transformers

     |████████████████████████████████| 4.2 MB 11.1 MB/s 
     |████████████████████████████████| 596 kB 48.0 MB/s 
     |████████████████████████████████| 84 kB 2.5 MB/s 
     |████████████████████████████████| 6.6 MB 45.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
import tensorflow as tf
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import re
from nltk import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
import nltk
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
nltk.download('wordnet')
# import dl_translate as dlt

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [6]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
  
tokenizerR = AutoTokenizer.from_pretrained("rabindralamsal/BERTsent")

modelR = TFAutoModelForSequenceClassification.from_pretrained("rabindralamsal/BERTsent")

Downloading:   0%|          | 0.00/323 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/929 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/515M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at rabindralamsal/BERTsent.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [ ]:
classifier = pipeline("sentiment-analysis")
classifier("I loved Star Wars so much!")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


[{'label': 'POSITIVE', 'score': 0.999840259552002}]

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("unicamp-dl/translation-pt-en-t5")
# model = AutoModelForSeq2SeqLM.from_pretrained("unicamp-dl/translation-pt-en-t5")
# pten_pipeline = pipeline('text2text-generation', model=model, tokenizer=tokenizer)
# mt = dlt.TranslationModel()  # Slow when you load it for the first time

In [7]:
example_tweet = "The NEET exams show our Govt in a poor light: unresponsiveness to genuine concerns; admit cards not delivered to aspirants in time; failure to provide centres in towns they reside, thus requiring unnecessary & risky travels. What a disgrace to treat our #Covid warriors like this!"
#this tweet resides on Twitter with an identifier-1435793872588738560
    
input = tokenizerR.encode(example_tweet, return_tensors="tf")
output = modelR.predict(input)[0]
prediction = tf.nn.softmax(output, axis=1).numpy()
sentiment = np.argmax(prediction)
    
print(prediction)
print(sentiment)

[[0.97267216 0.02368473 0.00364307]]
0


In [8]:
df_portuguese = pd.read_csv("./export_TweetSentBR.csv")
# df_english = pd.read_csv("datasets/export_TweetSentEnglish.csv")
# dataset = pd.read_csv("./datasetcompleto.csv")
completo = pd.read_csv("./datasetcompolaridade.csv")

In [9]:
completo.drop(columns = ['Unnamed: 0'], inplace = True)

In [ ]:
completo['tweets_translater_unicamp'] = completo['tweets_translater_unicamp'].replace('','')
completo['tweets_translater_face'] = completo['tweets_translater_face'].replace('"','')

In [10]:
df_portuguese

,id,id_twitter,text,sentiment
0,1343,863044774588272640,Que coisa linda! O Programa #encontro estava m...,1
1,1344,865583716088766467,"Por mais #Encontro com as Irmãs Galvão, adorei...",1
2,1345,865063232201011201,Mr. CATRA @OficialMrCatra lançando sua nova mú...,1
3,1346,864668391008763905,quem viu aquela lutadora modela barbuda tatuad...,0
4,1347,865572794016378882,Tô passada com esse cara.... quanta merda pode...,-1
...,...,...,...,...
11560,12908,864636619000877056,eu ja to aqui pronto pro #MasterChefBR mas ain...,-1
11561,12909,863581588713603072,MALUCO! Uma coisa que eu não tenho coragem é e...,-1
11562,12910,864831041349054464,#MaisVoce @ANAMARIABRAGA está linda @RedeGlobo,1
11563,12911,863042798575951872,"Que orgulho de ti, @sportrecife! #Encontro",1


In [11]:
df_remove = df_portuguese[df_portuguese['sentiment'] == '-']
df_portuguese = df_portuguese.drop(df_remove.index)
df_portuguese = df_portuguese.reset_index()
df_portuguese = df_portuguese.drop(columns=['index'])
df_portuguese['sentiment'] = df_portuguese['sentiment'].apply(lambda x: int(x))
Tweet = df_portuguese['text']
polarity = np.asarray(df_portuguese['sentiment'])

In [14]:
df_portuguese['text'] = df_portuguese['text'].apply(remove_user)
df_portuguese.drop(columns = ['id', 'id_twitter'], inplace = True)

In [16]:
completo

,tweets_original,tweets_translater_unicamp,tweets_translater_face,polaridade
0,Que coisa linda! O Programa #encontro estava m...,The #encontro Program was showing a family tha...,What a beautiful thing! The #touch program was...,1.0
1,"Por mais #Encontro com as Irmãs Galvão, adorei...",I loved them.,"For more I met with the Galvan Sisters, I love...",1.0
2,Mr. CATRA lançando sua nova música PPK CHORA ...,CATRA launching its new song PPK CHORA no k,Mr. CATRA releases his new song PPK CHORA on k...,1.0
3,quem viu aquela lutadora modela barbuda tatuad...,#MasterChefBR,Who saw that fighter model tattooed? #MasterCh...,0.0
4,Tô passada com esse cara.... quanta merda pode...,"I'm handled with this guy.... how mu""",How much shit can come out of someone’s mouth ...,-1.0
...,...,...,...,...
11523,a animação da filha da vanessa da mata é de se...,The animation of the daughter of vanessa da ma...,The animation of the daughter of the vanessa o...,1.0
11524,eu ja to aqui pronto pro #MasterChefBR mas ain...,"I'm already here ready for #MasterChefBR but""",I'm ready to go here for #MasterChefBR but I'm...,-1.0
11525,MALUCO! Uma coisa que eu não tenho coragem é e...,"One thing I don't have the courage is this""",One thing I don’t have the courage is that nor...,-1.0
11526,#MaisVoce está linda,#By You are beautiful #ByYou are beauti,#MaisVoce is beautiful,1.0


In [17]:
completo = completo.dropna().reset_index().drop(columns = ['index'])

In [13]:
def pre_X(frases):
    lista = []
    
    for frase in frases:
        lista.append(frase)
        
    return lista

def pre_Y(number):
    lista = []
    
    for numb in number:
        lista.append(numb)
    
    return lista


In [12]:
def set_array(frases):
    
    vocab = []
    palavras = []
    for frase in frases:
        
        text_array = remove_user(frase)
        text_array = Tokenize(text_array)
        text_array = text_array.split(' ')
        for i in range(len(text_array)):
            vocab.append(text_array[i])
    
    
        
    return vocab

def Tokenize(f):     ## Pre-processando a frase
    
    ## Colocando em minusculo
    ## Retirando a pontuaçao
    ## Retirando as StopWords
    
    f = f.lower().replace('\n', '').replace('-','').replace('#','').replace('.','').replace(',','').replace('!','').replace('r\n','').replace('  ','')
    token = RegexpTokenizer(r"\w+")
    f = token.tokenize(f)
    
    stop_words = set(stopwords.words('portuguese'))
    
    new_word = [word for word in f if not word in stop_words]
    
    return ' '.join(new_word)

def remove_user(frase):

    return re.sub('@\w+','',frase)

def translater_phrases(frase):
    translater = pten_pipeline(frase)
    translater = str(translater).strip('[{}]')
    translater = translater[19:]
    translater = translater.strip("''")
    
    return translater

def translater_frases(frase):
    text_hi = frase
    translater = mt.translate(text_hi, source=dlt.lang.PORTUGUESE, target=dlt.lang.ENGLISH)
    
    return translater

def new_classifier(frase):
    input = tokenizerR.encode(frase, return_tensors="tf")
    output = modelR.predict(input)[0]
    prediction = tf.nn.softmax(output, axis=1).numpy()
    sentiment = np.argmax(prediction)
    
    return sentiment 


In [ ]:
Tweet = Tweet.apply(remove_user)
# Tweet_preprocessed = Tweet.apply(Tokenize)

# count_vect = CountVectorizer()
# X_train = count_vect.fit_transform(Tweet_preprocessed)

# tfidf_transformer = TfidfTransformer()
# X_train_transform = tfidf_transformer.fit_transform(X_train) # Aplicando o TF-IDF


# X_train, X_test, Y_train, Y_test = train_test_split(X_train_transform, polarity, test_size=0.3)

NameError: name 'Tweet' is not defined

In [ ]:
classificacoes = pd.DataFrame(data = {'origem_portugues': df_portuguese['sentiment'], 
                                      'classificação_hug_com_unicamp_translater': completo['tweets_translater_unicamp'].apply(classifier),
                                      'classificação_rabin_com_unicamp_translater': completo['tweets_translater_unicamp'].apply(new_classifier),
                                      'classificação_hug_com_face_translater': completo['tweets_translater_face'].apply(classifier)})

In [18]:
classificacao = pd.DataFrame( data = { 'classificação_rabin_face_translater': completo['tweets_translater_face'].apply(new_classifier)})

In [19]:
classificacao

,classificação_rabin_face_translater
0,2
1,2
2,2
3,1
4,0
...,...
11505,1
11506,0
11507,0
11508,2


In [20]:
classificacao['classificação_rabin_face_translater'] = classificacao['classificação_rabin_face_translater'].replace(0, -1).replace(1,0).replace(2,1)

In [ ]:
classificacoes = classificacoes.dropna().reset_index().drop(columns = ['index'])

In [ ]:
classificacoes

,origem_portugues,classificação_hug_com_unicamp_translater,classificação_rabin_com_unicamp_translater,classificação_hug_com_face_translater
0,1,"[{'label': 'NEGATIVE', 'score': 0.978637635707...",1.0,"[{'label': 'POSITIVE', 'score': 0.999819815158..."
1,1,"[{'label': 'POSITIVE', 'score': 0.999877452850...",2.0,"[{'label': 'POSITIVE', 'score': 0.999851465225..."
2,1,"[{'label': 'POSITIVE', 'score': 0.993814468383...",1.0,"[{'label': 'NEGATIVE', 'score': 0.977949678897..."
3,0,"[{'label': 'POSITIVE', 'score': 0.983937680721...",1.0,"[{'label': 'NEGATIVE', 'score': 0.994238138198..."
4,-1,"[{'label': 'NEGATIVE', 'score': 0.977942943572...",1.0,"[{'label': 'NEGATIVE', 'score': 0.999268591403..."
...,...,...,...,...
11511,0,"[{'label': 'NEGATIVE', 'score': 0.854824185371...",1.0,"[{'label': 'POSITIVE', 'score': 0.999161243438..."
11512,-1,"[{'label': 'NEGATIVE', 'score': 0.973548531532...",1.0,"[{'label': 'NEGATIVE', 'score': 0.999184668064..."
11513,1,"[{'label': 'NEGATIVE', 'score': 0.999200165271...",0.0,"[{'label': 'NEGATIVE', 'score': 0.998830497264..."
11514,0,"[{'label': 'POSITIVE', 'score': 0.999661445617...",2.0,"[{'label': 'POSITIVE', 'score': 0.999835968017..."


In [ ]:
classificacoes['classificação_hug_com_unicamp_translater'] = classificacoes['classificação_hug_com_unicamp_translater'].apply(lambda x: x[0]['label'])
classificacoes['classificação_hug_com_face_translater'] = classificacoes['classificação_hug_com_face_translater'].apply(lambda x: x[0]['label'])

In [ ]:
classificacoes['classificação_hug_com_unicamp_translater'] = classificacoes['classificação_hug_com_unicamp_translater'].replace('POSITIVE', 1).replace('NEGATIVE',-1)
classificacoes['classificação_hug_com_face_translater'] = classificacoes['classificação_hug_com_face_translater'].replace('POSITIVE', 1).replace('NEGATIVE',-1)

In [ ]:
classificacoes['classificação_rabin_com_unicamp_translater'] = classificacoes['classificação_rabin_com_unicamp_translater'].replace(0, -1).replace(1,0).replace(2,1)
# classificacoes['classificação_rabin_face'] = classificacoes['classificação_rabin_face'].replace(0, -1).replace(1,0).replace(2,1)

In [ ]:
classificacoes

,origem_portugues,classificação_hug_com_unicamp_translater,classificação_rabin_com_unicamp_translater,classificação_hug_com_face_translater
0,1,-1,0.0,1
1,1,1,1.0,1
2,1,1,0.0,-1
3,0,1,0.0,-1
4,-1,-1,0.0,-1
...,...,...,...,...
11511,0,-1,0.0,1
11512,-1,-1,0.0,-1
11513,1,-1,-1.0,-1
11514,0,1,1.0,1


## Acuracia da origem com classificador do huggingface e traduzido com o tradutor da unicamp

In [21]:
soma = 0
for i in range (len(completo)):
      if(int(completo['polaridade'][i]) == int(classificacao['classificação_rabin_face_translater'][i])):
        soma+=1

soma/len(classificacao)

0.4774978279756733

## Acuracia da origem com classificador do rabin e traduzido com o tradutor da unicamp

In [ ]:
soma = 0
for i in range (len(classificacoes)):
    if(int(classificacoes['origem_portugues'][i]) == int(classificacoes['classificação_rabin_com_unicamp_translater'][i])):
        soma+=1

soma/len(classificacoes)

0.3109586662035429

## Acuracia da origem com classificador do huggingface e traduzido com o tradutor da m2m100 ou face

In [ ]:
soma = 0
for i in range (len(classificacoes)):
    if(int(classificacoes['origem_portugues'][i]) == int(classificacoes['classificação_hug_com_face_translater'][i])):
        soma+=1

soma/len(classificacoes)

0.37816950329975685

## Acuracia da origem com classificador do rabin e traduzido com o tradutor da m2m100 ou face

In [ ]:
soma = 0
for i in range (len(classificacoes)):
    if(int(classificacoes['origem_portugues'][i]) == int(classificacoes['classificação_rabin_face'][i])):
        soma+=1

soma/len(classificacoes)

0.65